# Tutorial: Auto Forecast

This notebook demonstrates zero-config ranking logic used by auto-forecast selection.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

from tollama.core.auto_select import build_auto_series_profile, select_auto_models
from tollama.core.schemas import SeriesInput

In [ ]:
timestamps = pd.date_range('2025-01-01', periods=40, freq='D')
series = [
    SeriesInput.model_validate({
        'id': 'region_A',
        'freq': 'D',
        'timestamps': timestamps.strftime('%Y-%m-%d').tolist(),
        'target': [100 + ((i % 7) * 2) + (i * 0.4) for i in range(40)],
        'past_covariates': {'promo': [1 if (i % 6 == 0) else 0 for i in range(40)]},
        'future_covariates': {'promo': [1, 0, 1, 0, 1, 0, 1]},
    })
]
profile = build_auto_series_profile(series=series)
profile

In [ ]:
include_models = ['mock', 'chronos2', 'granite-ttm-r2', 'timesfm-2.5-200m']
strategies = ['auto', 'fastest', 'best_accuracy', 'ensemble']
rank_rows = []
for strategy in strategies:
    selection = select_auto_models(
        series=series,
        horizon=7,
        strategy=strategy,
        include_models=include_models,
        ensemble_top_k=3,
    )
    for candidate in selection.ranked_candidates:
        rank_rows.append({
            'strategy': strategy,
            'model': candidate.model,
            'family': candidate.family,
            'score': candidate.score,
            'selected': candidate.model in selection.selected_models,
        })

ranking = pd.DataFrame(rank_rows)
ranking.head()

In [ ]:
auto_rank = ranking[ranking['strategy'] == 'auto'].sort_values('score', ascending=False)
fig, ax = plt.subplots(figsize=(9, 4))
ax.bar(auto_rank['model'], auto_rank['score'])
ax.set_title('Auto strategy ranking scores')
ax.set_ylabel('score')
ax.grid(axis='y', alpha=0.2)
plt.tight_layout()

In [ ]:
px.bar(
    ranking,
    x='model',
    y='score',
    color='strategy',
    barmode='group',
    title='Ranking by strategy',
)

In [ ]:
try:
    select_auto_models(series=series, horizon=7, strategy='auto', include_models=[])
except ValueError as exc:
    print('Error handling example:', exc)